In [33]:
import email
import getpass, imaplib
import os
import sys
from email.parser import HeaderParser
config = SecretConfig()
config.fromJsonFile()

In [24]:
detach_dir = '.'
if 'attachments' not in os.listdir(detach_dir):
    os.mkdir('attachments')

imapSession = imaplib.IMAP4_SSL('imap.gmail.com')
typ, accountDetails = imapSession.login(config.username, config.password)
if typ != 'OK':
    print ('Not able to sign in!')
    raise
print(accountDetails)

[b'home.assistant.sergey@gmail.com authenticated (Success)']


In [25]:
imapSession.select('camera/foscam')
typ, data = imapSession.search(None, 'ALL')
if typ != 'OK':
        print ('Error searching Inbox.')
        raise

In [46]:
msgId = data[0].split()[0]
typ, messageParts = imapSession.fetch(msgId, '(RFC822)')
emailBody = messageParts[0][1].decode('utf-8')
mail = email.message_from_string(emailBody)
import email.header
text, encoding = email.header.decode_header(mail['subject'])[0]
text.decode('utf-8')

'IPCamera alarm:FI9805W detected alarm at 2018-11-10 00:23:14'

In [ ]:
msgId = data[0].split()[0]
typ, messageParts = imapSession.fetch(msgId, '(RFC822)')
emailBody = messageParts[0][1].decode('utf-8')
mail = email.message_from_string(emailBody)
for part in mail.walk():
    print('=======================')
    if(part.get_content_maintype() != 'image'):
        continue
    fileName = part.get_filename()
    if bool(fileName):
        filePath = os.path.join(detach_dir, 'attachments', fileName)
        if not os.path.isfile(filePath) :
            print (filePath)
            fp = open(filePath, 'wb')
            fp.write(part.get_payload(decode=True))
            fp.close()

In [ ]:
def LoadLastMail():

    # Iterating over all emails
    for msgId in data[0].split(): 
        print('MsgId: ', msgId)
        typ, messageParts = imapSession.fetch(msgId, '(RFC822)')
        if typ != 'OK':
            print ('Error fetching mail.')
            raise

        emailBody = messageParts[0][1].decode('utf-8')
        mail = email.message_from_string(emailBody)

        for part in mail.walk():
            if part.get_content_maintype() == 'multipart':
                # print part.as_string()
                continue
            if part.get('Content-Disposition') is None:
                    # print part.as_string()
                continue
            fileName = part.get_filename()

            if bool(fileName):
                filePath = os.path.join(detach_dir, 'attachments', fileName)
                if not os.path.isfile(filePath) :
                    print (fileName)
                    #fp = open(filePath, 'wb')
                    #fp.write(part.get_payload(decode=True))
                    #fp.close()

In [ ]:
imapSession.close()
imapSession.logout()

In [ ]:
import json


class SecretConfig:
    usermane: str
    password: str
    filename = 'secret_config.json'
        
    def fromJsonFile(self):
        with open(self.filename, "r") as read_file:
            self.__dict__ = json.load(read_file)

    def fromJson(self, json_dump):
        self.__dict__ = json.loads(json_dump)

    def toJson(self):
        return json.dumps(self.__dict__, indent=4)

    def __repr__(self):
        return 'CONFIG: = {}'.format(self.filename)